In [1]:

from nltk.sentiment.vader import SentimentIntensityAnalyzer
import snscrape.modules.twitter as twitter
import pandas as pd
import itertools
import pysolr
import uuid
import nltk

nltk.download("vader_lexicon")
def crawl_from_twitter(cnt, query_term):
    sent_analyser = SentimentIntensityAnalyzer()
    tweets = twitter.TwitterSearchScraper(query_term).get_items()
    sliced_scraped_tweets = itertools.islice(tweets, 10)
    solr = pysolr.Solr(
        'http://localhost:8983/solr/stock_project_core', always_commit=True)
    tweets_to_add = []
    for tweet in sliced_scraped_tweets:
        sentiment = 0
        if sent_analyser.polarity_scores(tweet.rawContent)["compound"]>=0:
            sentiment = 1
        else:
            sentiment = 0
        temp = {
            "body": [tweet.rawContent],
            "id": str(uuid.uuid4()),
            "like_num": tweet.likeCount,
            "sentiment": sentiment,
            "ticker_symbol": query_term,
            "tweet_id": tweet.id,
            "post_date": str(tweet.date),
            "retweet_num": tweet.retweetCount,
            "writer": tweet.user.username,
        }
        tweets_to_add.append(temp)
    solr.add(tweets_to_add)
    return 
crawl_from_twitter(10, "tesla")

[nltk_data] Error loading vader_lexicon: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1131)>


In [3]:
df = pd.DataFrame(columns=['A','b'])

for i in range(5):
    new_row = pd.Series({'A': i, 'b': i-1})
    df=pd.concat([df, new_row.to_frame().T], ignore_index=True)
df


,A,b
0,0,-1
1,1,0
2,2,1
3,3,2
4,4,3


In [4]:
# , parallel_backend, register_parallel_backend
from joblib import Parallel, delayed
import numpy as np
import pandas as pd

import contractions
import textstat
import emoji as Emoji
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from textblob import TextBlob
# nltk.download('wordnet')
# nltk.download('omw-1.4')
# nltk.download('stopwords')
# nltk.download('vader_lexicon')
stop = stopwords.words('english')
# Parallel processing


class SubPreprocessor:
    def __init__(self, stemmer=PorterStemmer()):
        self.stemmer = stemmer
        #self.tokenize = tokenize
        self.stop_words = stop

    def fit(self, X, y=None, **fit_params):
        return self

    def transform(self, X, **transform_params):
        res = Parallel(n_jobs=-1)(
            delayed(self.processRow)(row[0]) for row in X.loc[:, 'raw_text']
        )
        res = pd.DataFrame(res, index=X.index)
        res = pd.concat([X, res], axis=1)
        return res

    def fit_transform(self, X, y=None, **fit_params):
        return self.fit(X, y).transform(X)

    def processRow(self, text):
        text, urls = self.extractLink(text, '')
        text, emojis = self.extractEmoji(text, '')
        text, emoticons = self.extractEmoticons(text, '')
        text = self.removePunctuation(text)
        text = self.removeRtLink(text)
        text = self.removeStopWords(text)

        return {
            #             'url_cnt': len(urls),
            #             'emoticons': emoticons,
            #             'emojis': emojis ,
            #             'emo_cnt': len(emoticons) + sum(emojis.values()),
            'clean_text_no_stem_user': text,
        }

    def removePunctuation(self, text, replace='', remove_num=False, remove_emoji=False):
        r_emoji = '\u00a9|\u00ae|[\u2000-\u3300]|\ud83c[\ud000-\udfff]|\ud83d[\ud000-\udfff]|\ud83e[\ud000-\udfff]'
        r = f'[!#"$%&\'()*+,-./:;<=>?@[\]^_`{{|}}~—]'
        if remove_emoji:
            r += f'|[{r_emoji}]'
        if remove_num:
            r += f'|[0-9]'
        text = re.sub(r, replace, text)
        return text

    def extractLink(self, text, replace_text=''):
        #r = '(http\S+?|www.\S+?)(?=\'|\")'
        #r = '(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b'
        #r = '''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))'''
        #r ='^https?:\/\/.*[\r\n]*'
        r = 'http://\S+|https://\S+'
        # .apply(lambda url: url[1:-1]) # trim quotes
        urls = re.findall(r, text)
        text = re.sub(r, replace_text, text)
        return text, urls

    def extractEmoticons(self, text, replace_text=''):
        r = '(?::|;|=|X)(?:-)?(?:\)|\(|D|P)'
        emoticons = re.findall(r, text)
        text = re.sub(r, replace_text, text)
        # replace('-','') removes nose of emoticons
        emoticons = [emoticon.replace('-', '') for emoticon in emoticons]
        return text, emoticons

    def extractEmoji(self, text, replace_text=''):
        distinct_ls = Emoji.distinct_emoji_list(text)
        emoji_cnt = dict(zip(distinct_ls, [0 for i in distinct_ls]))
        for emoji in emoji_cnt.keys():
            emoji_cnt[emoji] = text.count(emoji)
            text = text.replace(emoji, replace_text)

        return text, emoji_cnt

    def wordCount(self, text):
        return textstat.lexicon_count(text, removepunct=True)

    def splitWords(self, text):
        w_list = re.split('\s+', text.strip())
        return w_list

    def removeRtLink(self, text):
        # Removes RT
        #text = re.sub('RT @\w+: ','', text)
        #text = text.lower()

        # Removes @username from the tweet
        #text = re.sub(r'(@[A-Za-z0-9_]+)', '', text)

        # Removes link
        text = re.sub('http://\S+|https://\S+', '', text)

        # Only considers string or digits or whitespace
        text = re.sub(r'[^\w\s]', '', text)

        # Removes digits
        text = re.sub(r'\d+', '', text)
        return text

    def removeStopWords(self, text):
        #text_tokens = self.tokenize(text)
        text_tokens = word_tokenize(text)
        text = [word for word in text_tokens if not word in self.stop_words]
        return ' '.join(text)

    def __call__(self, text):
        return self.processRow(text)


In [5]:
import pandas as pd
df = pd.DataFrame(columns=['id', 'company', 'post_date', 'author',
                           'raw_text', 'like_num', 'subjectivity', 'sentiment'])
temp = {'id': 1642345902303219713, 'company': 'apple', 'post_date': '2023-04-02 01:59:11+00:00',
        'author': 'atomiccanonAPPL', 'raw_text': ['グレビュオフのコスプレできるやん！！！'], 'like_num': 0, 'subjectivity': 0, 'sentiment': 1}
df = pd.concat(
    [df, pd.DataFrame([temp], columns=df.columns)], ignore_index=True)
prep = SubPreprocessor()
df = prep.transform(df)
df


,id,company,post_date,author,raw_text,like_num,subjectivity,sentiment,clean_text_no_stem_user
0,1642345902303219713,apple,2023-04-02 01:59:11+00:00,atomiccanonAPPL,[グレビュオフのコスプレできるやん！！！],0,0,1,グレビュオフのコスプレできるやん


In [78]:
import pickle
import keras

# load the model from file
from keras.models import load_model
model = load_model('model.h5')


In [81]:
from textblob import TextBlob 
def is_subjective(text):
    blob = TextBlob(text)
    polarity = blob.sentiment.polarity
    subjectivity = blob.sentiment.subjectivity
    if subjectivity > 0.5:
        return 1
    return 0


def get_sentiment(text):
    blob = TextBlob(text)
    sentiment_score = blob.sentiment.polarity

    if sentiment_score < 0:
        return -1
    elif sentiment_score == 0:
        return 0
    else:
        return 1


print(is_subjective("BOT LOVE IS READY FOR WORLDWIDE LAUNCH starting tomorrow at 6:30AM pst! \U0001faf0🤖💗 \n\nYou all are going absolutely love it!🔥 \n\nGo Tesla Bot! ⚡️👊⚡️ \n\n$TSLA https://t.co/8oPD5VRNMv"))


0


In [57]:
import datetime
import pysolr
solr = pysolr.Solr(
    'http://localhost:8983/solr/stock_project_core', always_commit=True)
start_date = datetime.datetime(2023, 1, 1)
end_date = datetime.datetime(2023, 3, 30)
# Convert the dates to Solr date format
start_date_str = start_date.strftime('%Y-%m-%dT%H:%M:%SZ')
end_date_str = end_date.strftime('%Y-%m-%dT%H:%M:%SZ')
range_query = 'post_date:[{} TO {}]'.format(start_date_str, end_date_str)
t = "company:AAPL"
normal_params = {
    'q': t,
    'fq': range_query,
    'rows': 0,
}
results = solr.search(**normal_params)
total_tweets = results.hits

subjective_params = {
    'q': t + " AND subjectivity:1",
    'fq': range_query,
    'rows': 0,
}
print(subjective_params)
results = solr.search(**subjective_params)
subjective_tweets = results.hits

non_subjective_params = {
    'q': t + " AND subjectivity:0",
    'fq': range_query,
    'rows': 0,
}
print(non_subjective_params)
results = solr.search(**non_subjective_params)
non_subjective_tweets = results.hits

neutral_sentiment_params = {
    'q': t + " AND sentiment:0",
    'fq': range_query,
    'rows': 0,
}
print(neutral_sentiment_params)
results = solr.search(**neutral_sentiment_params)
neutral_tweets = results.hits

positive_sentiment_params = {
    'q': t + " AND sentiment:1",
    'fq': range_query,
    'rows': 0,
}

results = solr.search(**positive_sentiment_params)
positive_tweets = results.hits

negative_tweets = total_tweets - positive_tweets - neutral_tweets

non_sub_neu_params = {
    'q': t + " AND subjectivity:0 AND sentiment:0",
    'fq': range_query,
    'rows': 0,
}
results = solr.search(**non_sub_neu_params)
non_subjective_neutral = results.hits

non_sub_pos_params = {
    'q': t + " AND subjectivity:0 AND sentiment:1",
    'fq': range_query,
    'rows': 0,
}
results = solr.search(**non_sub_pos_params)
non_subjective_positive = results.hits
non_subjective_negative = non_subjective_tweets - non_subjective_positive - non_subjective_neutral

sub_neu_params = {
    'q': t + " AND subjectivity:1 AND sentiment:0",
    'fq': range_query,
    'rows': 0,
}
results = solr.search(**sub_neu_params)
subjective_neutral = results.hits

sub_pos_params = {
    'q': t + " AND subjectivity:1 AND sentiment:1",
    'fq': range_query,
    'rows': 0,
}
results = solr.search(**sub_pos_params)
subjective_positive = results.hits

subjective_negative = subjective_tweets - subjective_positive - subjective_neutral

response = {
    "total":total_tweets,
    "subjective": subjective_tweets,
    "non_subjective": non_subjective_tweets,
    "negative": negative_tweets,
    "neutral": neutral_tweets,
    "positive": positive_tweets,
    "non_subjective_negative": non_subjective_negative,
    "non_subjective_neutral": non_subjective_neutral,
    "non_subjective_positive": non_subjective_positive,
    "subjective_negative": subjective_negative,
    "subjective_neutral": subjective_neutral,
    "subjective_positive": subjective_positive
}
print(response)


{'q': 'company:AAPL AND subjectivity:1', 'fq': 'post_date:[2023-01-01T00:00:00Z TO 2023-03-30T00:00:00Z]', 'rows': 0}
{'q': 'company:AAPL AND subjectivity:0', 'fq': 'post_date:[2023-01-01T00:00:00Z TO 2023-03-30T00:00:00Z]', 'rows': 0}
{'q': 'company:AAPL AND sentiment:0', 'fq': 'post_date:[2023-01-01T00:00:00Z TO 2023-03-30T00:00:00Z]', 'rows': 0}
{'total': 892, 'subjective': 440, 'non_subjective': 452, 'negative': 111, 'neutral': 576, 'positive': 205, 'non_subjective_negative': 1, 'non_subjective_neutral': 441, 'non_subjective_positive': 10, 'subjective_negative': 110, 'subjective_neutral': 135, 'subjective_positive': 195}


In [77]:
from pysolr import Solr
solr = Solr('http://localhost:8983/solr/stock_project_core', always_commit=True)
query = solr.search("raw_text:love tim cokk", **{'rows': 20})
res = []
for i,v in enumerate(query.spellcheck["collations"]):
    if i % 2 == 0:
        continue
    else:
        s = v.replace("raw_text:", "")
        res.append(s)
print(res)

for res in query:
    print(res)


['love tim cook']
{'id': '1617303984942813184', 'company': ['TSLA'], 'post_date': '2023-01-22T23:31:32Z', 'author': ['EvasTeslaSPlaid'], 'raw_text': ['@cvalente28 Sorry. It’s done. I love $TSLA'], 'like_num': [64], 'subjectivity': [1], 'sentiment': [0], 'clean_text': 'cvalente28 Sorry Its done I love TSLA', '_version_': 1762527907934109696}
{'id': '1616793457481383936', 'company': ['GOOGL'], 'post_date': '2023-01-21T13:42:53Z', 'author': ['moyheen'], 'raw_text': ['Sending love to all my friends at the $goog ❤️'], 'like_num': [44], 'subjectivity': [1], 'sentiment': [0], 'clean_text': 'Sending love friends goog', '_version_': 1762527907537747972}
{'id': '1625622754690048000', 'company': ['AAPL'], 'post_date': '2023-02-14T22:27:21Z', 'author': ['sparkle6193920'], 'raw_text': ['Tomorrow $aapl gonna get sent 🚀 \n\nLove ❤️ ~ Your Uncle Buffet'], 'like_num': [55], 'subjectivity': [1], 'sentiment': [1], 'clean_text': 'Tomorrow aapl gon na get sent Love Your Uncle Buffet', '_version_': 17625279